In [4]:
!pip install "pandas>=1.5.3"
!pip install "numpy>=1.25.2"

In [5]:
import json
import pandas as pd
import operator
import numpy as np
from collections import Counter
f = open(f'sample/raw.json',"r",encoding="utf-8")
data = json.load(f)

Number of records:

In [6]:
print(len(data))

47


Filtering properties with empty values

In [7]:
for x in data:
  cleaned_empty_value=[]
  for y in x['properties']:
    if y['displayValue']!='':
      cleaned_empty_value.append(y)
  x['properties']=cleaned_empty_value

Load all properties to Pandas Dataframe

In [8]:
attrs=[]
vals=[]
for x in data:
  for y in x['properties']:
    attrs.append(y['displayName'])
    vals.append(y['displayValue'])


df_attr = pd.DataFrame({"attribute":attrs,"value":vals})
df_attr.groupby(['attribute',"value"]).size()

attribute          value                                        
12 Circuit Height  1.9166666666666665                               26
                   3.6666666666666665                               11
24 Circuit Height  2.1666666666666665                               26
                   3.6666666666666665                               11
30 Circuit Height  2.4166666666666665                               26
                                                                    ..
viewable_in        edb989ed-57fa-4318-88c9-6ba95b7b945f-000276db     2
                   edb989ed-57fa-4318-88c9-6ba95b7b945f-000276e2     5
                   f16d99ba-9667-4ad2-bb20-70f866953194-00167ffc     1
                   f16d99ba-9667-4ad2-bb20-70f866953194-0016802c     1
                   f16d99ba-9667-4ad2-bb20-70f866953194-00168035     4
Length: 1266, dtype: int64

Unique property keys

In [9]:
pd.DataFrame(df_attr['attribute'].unique())

,0
0,parent
1,viewable_in
2,Category
3,CategoryId
4,Level
...,...
93,12 Circuit Height
94,Load Classification
95,OmniClass Number
96,ADSK MS Comments


Number of unique values under each keys

In [10]:
df_attr.groupby('attribute')["value"].nunique().sort_values()

attribute
Lighting - Exterior Connected Current     1
Neutral Rating                            1
Neutral Bus                               1
Max Number of Circuits                    1
Distribution System                       1
                                         ..
Mark                                     47
ElementId                                47
Current Phase A                          47
IfcGUID                                  47
Panel Name                               47
Name: value, Length: 98, dtype: int64

Finding some attribute keys with medium distinct key size (manually select as >5 and <15)

In [11]:
df_fil = df_attr.groupby('attribute')["value"].nunique()

In [12]:
with pd.option_context('display.max_colwidth', None,
                       'display.max_columns', None,
                       'display.max_rows', None):
  display(df_attr[df_attr['attribute'].isin(df_fil[operator.and_(df_fil>5, df_fil <15)].index)].groupby('attribute')["value"].value_counts().to_frame())

value
attribute                              value                        
Circuit Number                         2                          10
                                       1                           9
                                       3                           9
                                       4                           7
                                       5                           3
                                       1,3,5                       1
                                       13,15,17                    1
                                       14,16,18                    1
                                       19,21,23                    1
                                       2,4,6                       1
                                       20,22,24                    1
                                       7,9,11                      1
                                       8,10,12                     1
Depth                                  0.4791666666666667         37
                                       0.5908203125                5
                                       0.6741536458333333          2
                                       0.9267578125                1
                                       1.0784505208333333          1
                                       3                           1
Elevation from Level                   4                          36
                                       4.739908854166716           3
                                       3.9065755208333837          2
                                       0.5416666666666679          1
                                       1.4999999999999982          1
                                       4.489908854166716           1
                                       4.75000000000005            1
                                       5.35742187500005            1
                                       5.46582031250005            1
HVAC Connected                         9999.990000000002          20
                                       19999.980000000003          8
                                       39999.96000000001           3
                                       49999.95000000001           2
                                       29999.97                    1
                                       59999.94000000002           1
                                       89999.91000000003           1
                                       129999.87000000005          1
                                       239999.7600000001           1
                                       369999.6300000002           1
HVAC Connected Current                 27.757196723045997         20
                                       55.514393446091994          8
                                       111.02878689218399          3
                                       138.78598361523             2
                                       83.271590169138             1
                                       166.54318033827602          1
                                       249.81477050741404          1
                                       360.84355739959807          1
                                       666.1727213531042           1
                                       1027.0162787527024          1
HVAC Estimated Demand                  9999.990000000002          20
                                       19999.980000000003          8
                                       39999.96000000001           3
                                       49999.95000000001           2
                                       29999.97                    1
                                       59999.94000000002           1
                                       89999.91000000003           1
                                       129999.87000000005          1
                                       239999.7600000001     

Finding attributes with all distinct values

In [13]:
df_attr.groupby('attribute')["value"].nunique().sort_values()[df_attr.groupby('attribute')["value"].nunique().sort_values()==len(data)]

attribute
Mark               47
ElementId          47
Current Phase A    47
IfcGUID            47
Panel Name         47
Name: value, dtype: int64

The "name" attribute from raw data can be considered as the category of objects

In [14]:
all_names=Counter()
for x in data:
  all_names[x['name']]+=1

for k,v in all_names.items():
  print(k,v)

Meter Main 2
Meter Bank 7
Switchboard 1
Lighting and Appliance Panelboard - 208V MCB 30
Lighting and Appliance Panelboard - 208V MCB - Surface 7


Finally the following attributes were selected for output
1. Mains
2. MCB Rating
3. Supply From: The upstream equipment (in Panel Name)
4. Panel Name: The name of equipment
5. Scedule Level/Level: physical location of the equipment

In [15]:
target_attr=("Mains","MCB Rating","Supply From","Panel Name","Schedule Level", "Level")
attr_with_unit=("Mains","MCB Rating")
to_df = []
for x in data:
  row = {}
  row['name']=x['name']
  for y in x['properties']:
    if y['displayName'] in target_attr:
      row[y['displayName']] = y['displayValue']
    if y['displayName'] in attr_with_unit:
      row[y['displayName']+"_unit"] = y['units']
  to_df.append(row)

In [16]:
df = pd.DataFrame.from_records(to_df)
df.head()

,name,Level,Mains,Mains_unit,MCB Rating,MCB Rating_unit,Supply From,Panel Name,Schedule Level
0,Meter Main,Parking,1600,autodesk.unit.unit:amperes-1.0.0,0,autodesk.unit.unit:amperes-1.0.0,SWITCHBOARD,MAIN1,NaN
1,Meter Main,Parking,1200,autodesk.unit.unit:amperes-1.0.0,0,autodesk.unit.unit:amperes-1.0.0,SWITCHBOARD,MAIN2,NaN
2,Meter Bank,Parking,0,autodesk.unit.unit:amperes-1.0.0,0,autodesk.unit.unit:amperes-1.0.0,MAIN1,MB11,NaN
3,Meter Bank,Parking,0,autodesk.unit.unit:amperes-1.0.0,0,autodesk.unit.unit:amperes-1.0.0,MAIN1,MB12,NaN
4,Meter Bank,Parking,0,autodesk.unit.unit:amperes-1.0.0,0,autodesk.unit.unit:amperes-1.0.0,MAIN2,MB21,NaN


Checking all records of Mains/MCB Rating have the same unit

In [17]:
print("Mains_unit: ",df['Mains_unit'].unique())
print("MCB Rating unit: ",df['MCB Rating_unit'].unique())

Mains_unit:  ['autodesk.unit.unit:amperes-1.0.0']
MCB Rating unit:  ['autodesk.unit.unit:amperes-1.0.0']


Adding unit (A) for Mains/MCB Rating

In [18]:
df = df.drop(columns=['Mains_unit',"MCB Rating_unit"])
df['Mains']=df['Mains'].apply(lambda x:str(x)+" A" if x!=0 else np.nan)
df['MCB Rating']=df['MCB Rating'].apply(lambda x:str(x)+" A" if x!=0 else np.nan)

In [19]:
df[['Panel Name','Mains','MCB Rating']].head(10)

,Panel Name,Mains,MCB Rating
0,MAIN1,1600 A,NaN
1,MAIN2,1200 A,NaN
2,MB11,NaN,NaN
3,MB12,NaN,NaN
4,MB21,NaN,NaN
5,MB22,NaN,NaN
6,MB23,NaN,NaN
7,MB14,NaN,NaN
8,MB13,NaN,NaN
9,SWITCHBOARD,3000 A,3000 A


The level of equipment were recorded in two different attributes (Level & Schedule Level)

In [20]:
df[['Level','Schedule Level']]

,Level,Schedule Level
0,Parking,NaN
1,Parking,NaN
2,Parking,NaN
3,Parking,NaN
4,Parking,NaN
5,Parking,NaN
6,Parking,NaN
7,Parking,NaN
8,Parking,NaN
9,Parking,NaN


No records have both "Level" and "Schedule Level" values

In [21]:
operator.and_((~df['Level'].isna()), (~df['Schedule Level'].isna())).all()

False

Merge the record from "Schedule Level" to "Level"

In [22]:
df["Level"] = df.apply(lambda x: (x["Schedule Level"] if pd.isnull(x["Level"]) else x["Level"]), axis=1)
df = df.drop(columns=['Schedule Level'])
df["Level"]

0           Parking
1           Parking
2           Parking
3           Parking
4           Parking
5           Parking
6           Parking
7           Parking
8           Parking
9           Parking
10               L5
11               L4
12               L3
13               L2
14               L5
15               L5
16               L4
17               L4
18               L3
19               L3
20               L2
21               L2
22               L2
23               L2
24               L4
25               L4
26               L4
27               L4
28               L3
29               L3
30               L3
31               L3
32               L4
33               L3
34               L5
35               L5
36               L4
37    L1 - Block 43
38    L1 - Block 43
39               L4
40    L1 - Block 37
41    L1 - Block 37
42    L1 - Block 37
43    L1 - Block 35
44               R2
45          Parking
46          Parking
Name: Level, dtype: object

In [23]:
df

,name,Level,Mains,MCB Rating,Supply From,Panel Name
0,Meter Main,Parking,1600 A,NaN,SWITCHBOARD,MAIN1
1,Meter Main,Parking,1200 A,NaN,SWITCHBOARD,MAIN2
2,Meter Bank,Parking,NaN,NaN,MAIN1,MB11
3,Meter Bank,Parking,NaN,NaN,MAIN1,MB12
4,Meter Bank,Parking,NaN,NaN,MAIN2,MB21
5,Meter Bank,Parking,NaN,NaN,MAIN2,MB22
6,Meter Bank,Parking,NaN,NaN,MAIN2,MB23
7,Meter Bank,Parking,NaN,NaN,MAIN1,MB14
8,Meter Bank,Parking,NaN,NaN,MAIN1,MB13
9,Switchboard,Parking,3000 A,3000 A,NaN,SWITCHBOARD


Transform the dataframe into nodes/edges information

In [24]:
df_res=df.reset_index()
df_res=df_res.rename({'index': 'id', 'name': 'group', 'Panel Name':'name','MCB Rating':"MCB_Rating"}, axis=1)

In [25]:
df_nodes=df_res.drop('Supply From', axis=1)

Final result of nodes dataframe

In [26]:
df_nodes.head()

,id,group,Level,Mains,MCB_Rating,name
0,0,Meter Main,Parking,1600 A,NaN,MAIN1
1,1,Meter Main,Parking,1200 A,NaN,MAIN2
2,2,Meter Bank,Parking,NaN,NaN,MB11
3,3,Meter Bank,Parking,NaN,NaN,MB12
4,4,Meter Bank,Parking,NaN,NaN,MB21


self-join the record based on the "Supply From" attribute

In [27]:
df_links=df_res.merge(df_res,left_on="Supply From",right_on="name")

In [28]:
df_links.head()

,id_x,group_x,Level_x,Mains_x,MCB_Rating_x,Supply From_x,name_x,id_y,group_y,Level_y,Mains_y,MCB_Rating_y,Supply From_y,name_y
0,0,Meter Main,Parking,1600 A,NaN,SWITCHBOARD,MAIN1,9,Switchboard,Parking,3000 A,3000 A,NaN,SWITCHBOARD
1,1,Meter Main,Parking,1200 A,NaN,SWITCHBOARD,MAIN2,9,Switchboard,Parking,3000 A,3000 A,NaN,SWITCHBOARD
2,2,Meter Bank,Parking,NaN,NaN,MAIN1,MB11,0,Meter Main,Parking,1600 A,NaN,SWITCHBOARD,MAIN1
3,3,Meter Bank,Parking,NaN,NaN,MAIN1,MB12,0,Meter Main,Parking,1600 A,NaN,SWITCHBOARD,MAIN1
4,7,Meter Bank,Parking,NaN,NaN,MAIN1,MB14,0,Meter Main,Parking,1600 A,NaN,SWITCHBOARD,MAIN1


Output the results of link in terms of id

In [29]:
df_links=df_links.rename({"id_x":"target","id_y":"source"}, axis=1)[["source","target"]]

Final result of links dataframe

In [30]:
df_links.head()

,source,target
0,9,0
1,9,1
2,0,2
3,0,3
4,0,7


Export the result to JSON file

In [31]:
df_nodes.fillna("N/A").to_json(r'df_nodes.json',orient='records')

In [32]:
df_links.to_json(r'df_links.json',orient='records')